In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         45           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        105           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        626            True  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        124           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1333            True  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14860            True  
house-pr

In [ ]:
!kaggle datasets download -d rishabkoul1/vechicle-dataset

  0% 0.00/6.21M [00:00<?, ?B/s]
100% 6.21M/6.21M [00:00<00:00, 141MB/s]


In [ ]:
!unzip "/content/vechicle-dataset.zip" -d "/content/drive/MyDrive/Kaggle Datasets"

In [ ]:
train_dir = "/content/drive/MyDrive/Kaggle Datasets/vechicle_with_train/vechicles/train"
test_dir = "/content/drive/MyDrive/Kaggle Datasets/vechicle_with_train/vechicles/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator = train_datagen.flow_from_directory(
    directory=train_dir, target_size=(150, 150), color_mode='rgb', class_mode='categorical', batch_size=32,
    shuffle=True, seed=2022)

Found 528 images belonging to 10 classes.


In [ ]:
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
validation_generator = test_datagen.flow_from_directory(
    directory=test_dir, target_size=(150, 150), color_mode='rgb', class_mode='categorical',
    batch_size=32, shuffle=True, seed=2022)

Found 74 images belonging to 10 classes.


In [ ]:
import tensorflow as tf

Model Definition

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='selu',
                          kernel_regularizer=tf.keras.regularizers.L1L2()),
    tf.keras.layers.Dense(512, activation='selu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,verbose=2,restore_best_weights=True)
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator,
                    epochs=50,verbose=2, callbacks=[monitor])

Epoch 1/50
17/17 - 218s - loss: 162.3212 - accuracy: 0.1061 - val_loss: 24.9403 - val_accuracy: 0.1486 - 218s/epoch - 13s/step
Epoch 2/50
17/17 - 65s - loss: 14.1022 - accuracy: 0.1439 - val_loss: 8.2552 - val_accuracy: 0.1622 - 65s/epoch - 4s/step
Epoch 3/50
17/17 - 59s - loss: 3.4282 - accuracy: 0.3106 - val_loss: 2.5998 - val_accuracy: 0.2838 - 59s/epoch - 3s/step
Epoch 4/50
17/17 - 60s - loss: 1.0748 - accuracy: 0.6629 - val_loss: 1.8645 - val_accuracy: 0.3919 - 60s/epoch - 4s/step
Epoch 5/50
17/17 - 62s - loss: 0.5145 - accuracy: 0.8428 - val_loss: 1.8114 - val_accuracy: 0.5000 - 62s/epoch - 4s/step
Epoch 6/50
17/17 - 60s - loss: 0.1720 - accuracy: 0.9583 - val_loss: 1.8954 - val_accuracy: 0.4865 - 60s/epoch - 4s/step
Epoch 7/50
17/17 - 61s - loss: 0.0630 - accuracy: 0.9943 - val_loss: 2.0011 - val_accuracy: 0.4595 - 61s/epoch - 4s/step
Epoch 8/50
17/17 - 61s - loss: 0.0230 - accuracy: 1.0000 - val_loss: 1.7864 - val_accuracy: 0.5135 - 61s/epoch - 4s/step
Epoch 9/50
17/17 - 60s - 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()